# Demonstration

This notebook presents two scripts to generate sample videos of how point tracking works.

In [7]:
import numpy as np
import cv2 as cv
import cv2

def fingerprint(path):
    '''
    Function to generate the path mask of a video

    path:     str: video path
    filename: str: name of the image to save
    dirr:     str: directory where you save the image
    
    '''
    cap = cv.VideoCapture(path)
    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                           qualityLevel = 0.3,
                           minDistance = 7,
                           blockSize = 7 )
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15, 15),
                      maxLevel = 2,
                      criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
    # Create color
    color = np.array([0,255,0])

    # Take first frame and find corners in it
    # Discard the first 3 frames (some videos started with a black or white frame, 
    # that prevented tracking and the function did not work)
    for j in range(3): 
        ret, old_frame = cap.read()
        old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

    p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

    #-----------
    height, width, _ = old_frame.shape
    ny = height/10
    nx = width/15
    
    # Point selecction
    p0 = np.array([[[ny,nx]]])

    t=0
    for j in range(15):
        t+=1
        n=0
        y = ny*t
        for i in range(10):
            n+=1
            x = nx*n
            p0 = np.append(p0,[np.array([[y,x]])],axis= 0)

    p0 = np.float32(p0)

    count=0
    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)
    counter = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        count+=1
        
    # Count frame n° 4
        if count > 3:
            frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
      
        # Calculate optical flow
            p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
      
        # Select good points
            if p1 is not None:
                good_new = p1[st==1]
                good_old = p0[st==1]

        # draw the tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()

                try:
                    mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color.tolist(), 1, lineType = 50) 
                    frame = cv.circle(frame, (int(a), int(b)), 5, color.tolist(), -1) 

                except:
                    pass

            img2 = cv.add(frame, mask)
            
            cv2.imwrite("./probe/0000" + str(counter) + ".jpg", img2)

        # Now update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

            counter += 1

In [14]:
# Script to convert video to video maskara

import cv2
import os

def video_reverse(pathing, save, name):

    '''pathing: str: directory of the video to invert
       save:    str: directory to save the generated video
       name:    str: file name '''
    
    def walkdir(folder):
        ''' Firectories function '''
        for dirpath, _, files in os.walk(folder):
            for filename in files:
                yield (dirpath, filename)


    # Generate the video frames with mask in the probe folder

    fingerprint(pathing) 

    # --------------------------------------

    # Empty array
    img_array = []

    # Generate list with frames path
    f = r'./probe/'
    li = []
    for dir,filename in walkdir(f):
        li.append(dir+filename)

    # Generate a new list in reverse order
    list_in_reverse=[]    
    for q in range(len(li)):
        
        mas = 0
        for j in range(len(li)):
            n = int(li[j][len(li[j])-8:len(li[j])-4])
            
            if n >= mas:
                mas = n
                r = li[j]
        list_in_reverse.append(r)
        li.remove(r)

    # To reverse the order or not   
    list_in_reverse.reverse()

    # "For" to read images from a directory
    for p in list_in_reverse:
        path = p
        img = cv2.imread(path)
        img_array.append(img)

    # Size of the last image height and width
    height, width  = img.shape[:2]
    # "name" is the name of the file and "save" directory, everything is a "str"
    video = cv2.VideoWriter(save + name ,cv2.VideoWriter_fourcc(*'mp4v'), 24,(width, height))

    # "For" to save frames in a video
    for i in range(len(img_array)):
        video.write(img_array[i])

    video.release()  

    # Remove the frames
    for dir, filename in walkdir('./probe'):
        fol = str(dir + '/' + filename)
        os.remove(fol)
        
        
video_reverse('./video_example/static.mp4', './', 'tracking_point_static.mp4');